In [1]:
import pickle
import numpy as np

ising_data = pickle.load(open("../data/ising/Ising2DFM_reSample_L40_T=All.pkl", "rb"))
ising_data = np.unpackbits(ising_data).reshape(-1, 1600)
ising_data = ising_data.astype("int")
ising_data[np.where(ising_data == 0)] = -1

T = np.linspace(0.25, 4.0, 16)
T_C = 2.26
labels = np.ones((16, 10000), dtype=np.int8)

for i, T_ in enumerate(T):
    if T_ > T_C:
        labels[i] = np.zeros(10000, dtype=np.int8)
labels = labels.reshape(16 * 10000)


In [2]:
from sklearn.model_selection import train_test_split

train_to_test_ratio = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(ising_data, labels, train_size=train_to_test_ratio)


/home/andreas/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
from scipy.special import expit

class BinaryNeuralNetwork:
    def __init__(
        self,
        X_data,
        Y_data,
        n_neurons_layer1=100,
        n_neurons_layer2=50,
        n_categories=2,
        epochs=1,
        batch_size=100,
        eta=0.1,
        lmbd=0.0,
    ):
        self.X_data_full = X_data
        self.Y_data_full = Y_data

        self.n_inputs = X_data.shape[0]
        self.n_features = X_data.shape[1]
        self.n_neurons_layer1 = n_neurons_layer1
        self.n_neurons_layer2 = n_neurons_layer2
        self.n_categories = n_categories

        self.epochs = epochs
        self.batch_size = batch_size
        self.iterations = self.n_inputs // self.batch_size
        self.eta = eta
        self.lmbd = lmbd

        self.create_biases_and_weights()

    def create_biases_and_weights(self):
        self.weights_layer1 = np.random.randn(self.n_features, self.n_neurons_layer1)
        self.bias_layer1 = np.zeros(self.n_neurons_layer1)

        self.weights_layer2 = np.random.randn(self.n_neurons_layer1, self.n_neurons_layer2)
        self.bias_layer2 = np.zeros(self.n_neurons_layer2)

        self.weights_output = np.random.randn(self.n_neurons_layer2, self.n_categories)
        self.bias_output = np.zeros(self.n_categories)

    def feed_forward(self):
        self.z1 = np.dot(self.X_data, self.weights_layer1) + self.bias_layer1
        self.a1 = expit(self.z1)

        self.z2 = np.dot(self.a1, self.weights_layer2) + self.bias_layer2
        self.a2 = expit(self.z2)

        self.z3 = np.dot(self.a2, self.weights_output) + self.bias_output

        exp_term = np.exp(self.z3)
        self.probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

    def feed_forward_out(self, X):
        z1 = np.dot(X, self.weights_layer1) + self.bias_layer1
        a1 = expit(z1)

        z2 = np.dot(a1, self.weights_layer2) + self.bias_layer2
        a2 = expit(z2)

        z3 = np.dot(a2, self.weights_output) + self.bias_output

        exp_term = np.exp(z3)
        probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)
        return probabilities

    def backpropagation(self):
        error_output = self.probabilities
        error_output[range(self.n_inputs), self.Y_data] -= 1
        error_layer2 = np.dot(error_output, self.weights_output.T) * self.a2 * (1 - self.a2)
        error_layer1 = np.dot(error_layer2, self.weights_layer2.T) * self.a1 * (1 - self.a1)

        self.weights_output_gradient = np.dot(self.a2.T, error_output)
        self.bias_output_gradient = np.sum(error_output)

        self.weights_layer2_gradient = np.dot(self.a1.T, error_layer2)
        self.bias_layer2_gradient = np.sum(error_layer2)

        self.weights_layer1_gradient = np.dot(self.X_data.T, error_layer1)
        self.bias_layer1_gradient = np.sum(error_layer1)

        if self.lmbd > 0.0:
            self.weights_output_gradient += self.lmbd * self.weights_output
            self.weights_layer2_gradient += self.lmbd * self.weights_layer2
            self.weights_layer1_gradient += self.lmbd * self.weights_layer1

        self.weights_output -= self.eta * self.weights_output_gradient
        self.bias_output -= self.eta * self.bias_output_gradient
        self.weights_layer2 -= self.eta * self.weights_layer2_gradient
        self.bias_layer2 -= self.eta * self.bias_layer2_gradient
        self.weights_layer1 -= self.eta * self.weights_layer1_gradient
        self.bias_layer1 -= self.eta * self.bias_layer1_gradient

    def predict(self, X):
        probabilities = self.feed_forward_out(X)
        return np.argmax(probabilities, axis=1)

    def predict_probabilities(self, X):
        probabilities = self.feed_forward_out(X)
        return probabilities

    def train(self):
        data_indices = np.arange(self.n_inputs)

        for i in range(self.epochs):
            for j in range(self.iterations):
                chosen_datapoints = np.random.choice(
                    data_indices, size=self.batch_size, replace=False
                )

                self.X_data = self.X_data_full[chosen_datapoints]
                self.Y_data = self.Y_data_full[chosen_datapoints]

                self.n_inputs = self.X_data.shape[0]

                self.feed_forward()
                self.backpropagation()


In [4]:
from sklearn.metrics import accuracy_score

epochs = 10
batch_size = 100
n_neurons_layer1 = 100
n_neurons_layer2 = 50
n_categories = 2

eta_vals = np.logspace(-5, 0, 6)
lmbd_vals = np.logspace(-5, 0, 6)

In [5]:
DNN_numpy = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)

for i, eta in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals):
        dnn = BinaryNeuralNetwork(X_train, Y_train, eta=eta, lmbd=lmbd, epochs=epochs, batch_size=batch_size,
                                  n_neurons_layer1=n_neurons_layer1, n_neurons_layer2=n_neurons_layer2,
                                  n_categories=n_categories)
        dnn.train()
        
        DNN_numpy[i][j] = dnn
        
        test_predict = dnn.predict(X_test)
        
        print("Learning rate  = ", eta)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", accuracy_score(Y_test, test_predict))
        print()

Learning rate  =  1e-05
Lambda =  1e-05
Accuracy score on test set:  0.90625

Learning rate  =  1e-05
Lambda =  0.0001
Accuracy score on test set:  0.8994375

Learning rate  =  1e-05
Lambda =  0.001
Accuracy score on test set:  0.9193125

Learning rate  =  1e-05
Lambda =  0.01
Accuracy score on test set:  0.87525

Learning rate  =  1e-05
Lambda =  0.1
Accuracy score on test set:  0.91475

Learning rate  =  1e-05
Lambda =  1.0
Accuracy score on test set:  0.90790625

Learning rate  =  0.0001
Lambda =  1e-05
Accuracy score on test set:  0.97753125

Learning rate  =  0.0001
Lambda =  0.0001
Accuracy score on test set:  0.9770625

Learning rate  =  0.0001
Lambda =  0.001
Accuracy score on test set:  0.97771875

Learning rate  =  0.0001
Lambda =  0.01
Accuracy score on test set:  0.9785625

Learning rate  =  0.0001
Lambda =  0.1
Accuracy score on test set:  0.97725

Learning rate  =  0.0001
Lambda =  1.0
Accuracy score on test set:  0.9900625

Learning rate  =  0.001
Lambda =  1e-05
Accurac

In [6]:
from sklearn.neural_network import MLPClassifier

DNN_scikit = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)

for i, eta in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals):
        dnn = MLPClassifier(hidden_layer_sizes=(n_neurons_layer1, n_neurons_layer2), activation='logistic',
                            alpha=lmbd, learning_rate_init=eta, max_iter=100)
        dnn.fit(X_train, Y_train)
        
        DNN_scikit[i][j] = dnn
        
        print("Learning rate  = ", eta)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", dnn.score(X_test, Y_test))
        print()

/home/andreas/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Learning rate  =  1e-05
Lambda =  1e-05
Accuracy score on test set:  0.99246875

Learning rate  =  1e-05
Lambda =  0.0001
Accuracy score on test set:  0.9923125

Learning rate  =  1e-05
Lambda =  0.001
Accuracy score on test set:  0.9929375

Learning rate  =  1e-05
Lambda =  0.01
Accuracy score on test set:  0.9931875

Learning rate  =  1e-05
Lambda =  0.1
Accuracy score on test set:  0.99275

Learning rate  =  1e-05
Lambda =  1.0
Accuracy score on test set:  0.99271875

Learning rate  =  0.0001
Lambda =  1e-05
Accuracy score on test set:  0.9935625

Learning rate  =  0.0001
Lambda =  0.0001
Accuracy score on test set:  0.99365625

Learning rate  =  0.0001
Lambda =  0.001
Accuracy score on test set:  0.994

Learning rate  =  0.0001
Lambda =  0.01
Accuracy score on test set:  0.99475

Learning rate  =  0.0001
Lambda =  0.1
Accuracy score on test set:  0.9951875

Learning rate  =  0.0001
Lambda =  1.0
Accuracy score on test set:  0.99371875

Learning rate  =  0.001
Lambda =  1e-05
Accura